# 🚀 PPO Farm (Batch Mode)
Train **multiple models** with randomized configs in one run.

In [ ]:
# Setup (quiet)
!pip install stable-baselines3[extra] gymnasium pandas pyarrow -q
import warnings; warnings.filterwarnings('ignore')

from google.colab import drive, files
drive.mount('/content/drive')

import os, json, time, shutil
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
import gymnasium as gym
from gymnasium import spaces

OUTPUT = '/content/drive/MyDrive/NeuroTrader_Models'
os.makedirs(OUTPUT, exist_ok=True)
print(f"✅ Ready | GPU: {torch.cuda.is_available()}")

In [ ]:
# Upload data
os.makedirs('data/raw', exist_ok=True)
print("📤 Upload XAUUSDm_M5_raw.parquet:")
for f in files.upload().keys():
    shutil.move(f, f'data/raw/{f}')
    print(f"✅ {f}")

In [ ]:
#@title ⚙️ Config
NUM_MODELS = 5 #@param {type:"slider", min:1, max:10, step:1}
STEPS_PER_MODEL = 300000 #@param {type:"slider", min:100000, max:1000000, step:100000}
NUM_ENVS = 4 #@param {type:"slider", min:1, max:8, step:1}

print(f"📋 Will train {NUM_MODELS} models × {STEPS_PER_MODEL:,} steps")
print(f"   Using {NUM_ENVS} parallel environments per model")

In [ ]:
# Environment
class TradingEnv(gym.Env):
    def __init__(self, df):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.idx = 20
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(-np.inf, np.inf, (22,), np.float32)
        self.pos = self.entry = 0
        self.bal = 10000
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.idx = 20; self.pos = self.entry = 0; self.bal = 10000
        return self._obs(), {}
    
    def _obs(self):
        c = self.df.iloc[self.idx-20:self.idx]['close'].values
        r = np.diff(c)/(c[:-1]+1e-8)
        return np.concatenate([r[-10:],[c[-1]/c[0]-1],[(c[-1]-c.min())/(c.max()-c.min()+1e-8)],[np.std(r)],[self.pos],[self.bal/10000-1],np.zeros(7)]).astype(np.float32)
    
    def step(self, a):
        p = self.df.iloc[self.idx]['close']; r = 0
        if a==1 and self.pos==0: self.pos=1; self.entry=p; r=0.01
        elif a==2 and self.pos==1: pnl=(p-self.entry)/self.entry; self.bal*=(1+pnl); r=pnl*100; self.pos=0
        self.idx += 1
        eq = self.bal*(1+(p-self.entry)/self.entry) if self.pos else self.bal
        return self._obs(), r, self.idx>=len(self.df)-1, eq<5000, {'eq':eq}

# Load data once
DF = pd.read_parquet('data/raw/XAUUSDm_M5_raw.parquet')
if 'tick_volume' in DF.columns: DF.rename(columns={'tick_volume':'volume'},inplace=True)
print(f"✅ {len(DF)} rows")

In [ ]:
# Batch Training
def make_env(): return TradingEnv(DF)
results = []

print(f"\n🚀 Starting batch training...\n")
for i in range(NUM_MODELS):
    seed = int(time.time()*1000)%1000000
    np.random.seed(seed)
    
    cfg = {
        'lr': np.random.choice([1e-4,3e-4,5e-4,1e-3]),
        'n_steps': int(np.random.choice([512,1024,2048])),
        'ent': round(np.random.uniform(0.01,0.1),3),
        'gamma': np.random.choice([0.95,0.97,0.99])
    }
    
    env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])
    model = PPO("MlpPolicy", env, device='cuda', verbose=0, seed=seed,
                learning_rate=cfg['lr'], n_steps=cfg['n_steps'],
                ent_coef=cfg['ent'], gamma=cfg['gamma'])
    
    t0 = time.time()
    model.learn(STEPS_PER_MODEL, progress_bar=True)
    dur = (time.time()-t0)/60
    
    # Quick eval
    env_t = TradingEnv(DF[-5000:])
    obs,_ = env_t.reset()
    for _ in range(4000):
        a,_ = model.predict(obs,deterministic=True)
        obs,_,d,t,info = env_t.step(a)
        if d or t: break
    ret = (info['eq']/10000-1)*100
    
    # Save
    name = f"ppo_{i+1:02d}_s{seed}"
    model.save(f"{OUTPUT}/{name}")
    results.append({'name':name,'ret':ret,'cfg':cfg,'min':dur})
    env.close()
    
    print(f"[{i+1}/{NUM_MODELS}] {name}: {ret:+.1f}% ({dur:.1f}m)")

# Save log
with open(f"{OUTPUT}/batch_log.json",'w') as f: json.dump(results,f,indent=2)
print(f"\n✅ Done! {len(results)} models saved.")

In [ ]:
# Summary
print("\n📊 Results:")
for r in sorted(results,key=lambda x:x['ret'],reverse=True):
    print(f"  {r['name']}: {r['ret']:+.1f}%")